# 01 前情摘要

前面的task2与task3讲解了音频数据的分析以及特征提取等内容，本次任务主要是讲解CNN模型的搭建与训练，由于模型训练需要用到之前的特侦提取等得让，于是在此再贴一下相关代码。

## 1.1 导包

In [1]:
#基本库
import pandas as pd
import numpy as np
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler

#深度学习框架
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from tensorflow.keras.utils import to_categorical 
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import tensorflow as tf
import tensorflow.keras

#音频处理库
import os
import librosa
import librosa.display
import glob 

## 1.2 特征提取以及数据集的建立

In [2]:
feature = []
label = []
# 建立类别标签，不同类别对应不同的数字。
label_dict = {'aloe': 0, 'burger': 1, 'cabbage': 2,'candied_fruits':3, 'carrots': 4, 'chips':5,
                  'chocolate': 6, 'drinks': 7, 'fries': 8, 'grapes': 9, 'gummies': 10, 'ice-cream':11,
                  'jelly': 12, 'noodles': 13, 'pickles': 14, 'pizza': 15, 'ribs': 16, 'salmon':17,
                  'soup': 18, 'wings': 19}
label_dict_inv = {v:k for k,v in label_dict.items()}

建立提取音频特征的函数

In [3]:
from tqdm import tqdm
def extract_features(parent_dir, sub_dirs, max_file=10, file_ext="*.wav"):
    c = 0
    label, feature = [], []
    for sub_dir in sub_dirs:
        for fn in tqdm(glob.glob(os.path.join(parent_dir, sub_dir, file_ext))[:max_file]): # 遍历数据集的所有文件
            
           # segment_log_specgrams, segment_labels = [], []
            #sound_clip,sr = librosa.load(fn)
            #print(fn)
            label_name = fn.split('/')[-2]
            label.extend([label_dict[label_name]])
            X, sample_rate = librosa.load(fn,res_type='kaiser_fast')
            mels = np.mean(librosa.feature.melspectrogram(y=X,sr=sample_rate).T,axis=0) # 计算梅尔频谱(mel spectrogram),并把它作为特征
            feature.extend([mels])
            
    return [feature, label]

In [4]:
# 自己更改目录
parent_dir = './Task1_CNN_Baseline_1/train_sample/'
save_dir = "./"
folds = sub_dirs = np.array(['aloe','burger','cabbage','candied_fruits',
                             'carrots','chips','chocolate','drinks','fries',
                            'grapes','gummies','ice-cream','jelly','noodles','pickles',
                            'pizza','ribs','salmon','soup','wings'])

# 获取特征feature以及类别的label
temp = extract_features(parent_dir,sub_dirs,max_file=100)

100%|██████████| 35/35 [00:05<00:00,  6.14it/s]


In [5]:
temp

[[array([9.59090614e+00, 8.92415428e+00, 5.41957951e+00, 3.29546285e+00,
         2.37589765e+00, 1.90731287e+00, 1.81245482e+00, 1.69746757e+00,
         3.05571866e+00, 5.17213821e+00, 7.56661224e+00, 8.00160027e+00,
         6.89815521e+00, 3.18521214e+00, 1.46564400e+00, 9.97942924e-01,
         1.04116511e+00, 9.01841342e-01, 7.27221251e-01, 6.78908587e-01,
         8.11331511e-01, 6.04100466e-01, 4.78991270e-01, 4.20743972e-01,
         4.01466846e-01, 3.06602359e-01, 3.10562551e-01, 6.99183106e-01,
         5.80164254e-01, 4.87972140e-01, 4.84577894e-01, 4.75131840e-01,
         4.29968357e-01, 3.28041196e-01, 3.59113306e-01, 3.86157542e-01,
         6.74311399e-01, 1.10365069e+00, 1.87180853e+00, 2.92095447e+00,
         2.26343679e+00, 3.13818312e+00, 3.33579183e+00, 1.90766692e+00,
         1.96087301e+00, 2.06493020e+00, 1.98704290e+00, 1.58575881e+00,
         1.01975489e+00, 7.48212278e-01, 3.92245293e-01, 3.56836289e-01,
         2.80648530e-01, 2.04742327e-01, 1.91017896

In [6]:
temp = np.array(temp)
data = temp.transpose()

In [7]:
temp

array([[array([9.59090614e+00, 8.92415428e+00, 5.41957951e+00, 3.29546285e+00,
       2.37589765e+00, 1.90731287e+00, 1.81245482e+00, 1.69746757e+00,
       3.05571866e+00, 5.17213821e+00, 7.56661224e+00, 8.00160027e+00,
       6.89815521e+00, 3.18521214e+00, 1.46564400e+00, 9.97942924e-01,
       1.04116511e+00, 9.01841342e-01, 7.27221251e-01, 6.78908587e-01,
       8.11331511e-01, 6.04100466e-01, 4.78991270e-01, 4.20743972e-01,
       4.01466846e-01, 3.06602359e-01, 3.10562551e-01, 6.99183106e-01,
       5.80164254e-01, 4.87972140e-01, 4.84577894e-01, 4.75131840e-01,
       4.29968357e-01, 3.28041196e-01, 3.59113306e-01, 3.86157542e-01,
       6.74311399e-01, 1.10365069e+00, 1.87180853e+00, 2.92095447e+00,
       2.26343679e+00, 3.13818312e+00, 3.33579183e+00, 1.90766692e+00,
       1.96087301e+00, 2.06493020e+00, 1.98704290e+00, 1.58575881e+00,
       1.01975489e+00, 7.48212278e-01, 3.92245293e-01, 3.56836289e-01,
       2.80648530e-01, 2.04742327e-01, 1.91017896e-01, 1.99287936e-01

In [8]:
data

array([[array([9.59090614e+00, 8.92415428e+00, 5.41957951e+00, 3.29546285e+00,
       2.37589765e+00, 1.90731287e+00, 1.81245482e+00, 1.69746757e+00,
       3.05571866e+00, 5.17213821e+00, 7.56661224e+00, 8.00160027e+00,
       6.89815521e+00, 3.18521214e+00, 1.46564400e+00, 9.97942924e-01,
       1.04116511e+00, 9.01841342e-01, 7.27221251e-01, 6.78908587e-01,
       8.11331511e-01, 6.04100466e-01, 4.78991270e-01, 4.20743972e-01,
       4.01466846e-01, 3.06602359e-01, 3.10562551e-01, 6.99183106e-01,
       5.80164254e-01, 4.87972140e-01, 4.84577894e-01, 4.75131840e-01,
       4.29968357e-01, 3.28041196e-01, 3.59113306e-01, 3.86157542e-01,
       6.74311399e-01, 1.10365069e+00, 1.87180853e+00, 2.92095447e+00,
       2.26343679e+00, 3.13818312e+00, 3.33579183e+00, 1.90766692e+00,
       1.96087301e+00, 2.06493020e+00, 1.98704290e+00, 1.58575881e+00,
       1.01975489e+00, 7.48212278e-01, 3.92245293e-01, 3.56836289e-01,
       2.80648530e-01, 2.04742327e-01, 1.91017896e-01, 1.99287936e-01

In [9]:
# 获取特征
X = np.vstack(data[:, 0])

# 获取标签
Y = np.array(data[:, 1])
print('X的特征尺寸是：',X.shape)
print('Y的特征尺寸是：',Y.shape)

X的特征尺寸是： (1000, 128)
Y的特征尺寸是： (1000,)


In [10]:
# 在Keras库中：to_categorical就是将类别向量转换为二进制（只有0和1）的矩阵类型表示
Y = to_categorical(Y)

In [11]:
'''最终数据'''
print(X.shape)
print(Y.shape)

(1000, 128)
(1000, 20)


In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1, stratify=Y)
print('训练集的大小',len(X_train))
print('测试集的大小',len(X_test))

训练集的大小 750
测试集的大小 250


In [13]:
X_train = X_train.reshape(-1, 16, 8, 1)
X_test = X_test.reshape(-1, 16, 8, 1)

# 02 建立模型

## 2.1 深度学习框架

Keras 是一个用 Python 编写的高级神经网络 API，它能够以 TensorFlow, CNTK, 或者 Theano 作为后端运行。现在Keras已经和TensorFlow合并，可以通过TensorFlow来调用。

### 2.1.1 网络结构搭建

Keras 的核心数据结构是 model，一种组织网络层的方式。最简单的模型是 Sequential 顺序模型，它由多个网络层线性堆叠。对于更复杂的结构，你应该使用 Keras 函数式 API，它允许构建任意的神经网络图。

Sequential模型可以直接通过如下方式搭建：

from keras.models import Sequential

model = Sequential()

In [14]:
model = Sequential()

### 2.1.2 搭建CNN网络

In [15]:
# 输入的大小
input_dim = (16, 8, 1)

### 2.1.3 CNN基础知识

> 推荐的资料中，我们推荐大家去看看李宏毅老师的讲的[CNN网络](https://www.youtube.com/watch?v=FrKWiRv254g&feature=youtu.be)这里也附上老师的[PPT](http://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2017/Lecture/CNN.pdf)。

CNN网络的基本架构

![图片.png](attachment:28e6803d-dac4-4e48-b51f-a6b84545955e.png)

卷积神经网络CNN的结构一般包含这几个层：

1)输入层：用于数据的输入

2)卷积层：使用卷积核进行特征提取和特征映射------>可以多次重复使用

3)激励层：由于卷积也是一种线性运算，因此需要增加非线性映射(也就是激活函数)

4)池化层：进行下采样，对特征图稀疏处理，减少数据运算量----->可以多次重复使用

5）Flatten操作：将二维的向量，拉直为一维的向量，从而可以放入下一层的神经网络中

6)全连接层：通常在CNN的尾部进行重新拟合，减少特征信息的损失----->DNN网络

对于Keras操作中，可以简单地使用 `.add()` ，将需要搭建的神经网络的layer堆砌起来，像搭积木一样：

In [16]:
model.add(Conv2D(64, (3, 3), padding = "same", activation = "tanh", input_shape = input_dim))# 卷积层
model.add(MaxPool2D(pool_size=(2, 2)))# 最大池化
model.add(Conv2D(128, (3, 3), padding = "same", activation = "tanh")) #卷积层
model.add(MaxPool2D(pool_size=(2, 2))) # 最大池化层
model.add(Dropout(0.1))
model.add(Flatten()) # 展开
model.add(Dense(1024, activation = "tanh"))
model.add(Dense(20, activation = "softmax")) # 输出层：20个units输出20个类的概率

如果需要，你还可以进一步地配置你的优化器`.complies()`)。Keras 的核心原则是使事情变得相当简单，同时又允许用户在需要的时候能够进行完全的控制（终极的控制是源代码的易扩展性）。

In [17]:
# 编译模型，设置损失函数，优化方法以及评价标准
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# 03 CNN模型训练与测试

## 3.1 模型训练
批量的在之前搭建的模型上训练：

In [18]:
# 训练模型
model.fit(X_train, Y_train, epochs = 90, batch_size = 50, validation_data = (X_test, Y_test))

Train on 750 samples, validate on 250 samples
Epoch 1/90
750/750 [==============================] - 27s 35ms/sample - loss: 2.8646 - accuracy: 0.1067 - val_loss: 2.7060 - val_accuracy: 0.1440
Epoch 2/90
750/750 [==============================] - 0s 160us/sample - loss: 2.5083 - accuracy: 0.2467 - val_loss: 2.5323 - val_accuracy: 0.2280
Epoch 3/90
750/750 [==============================] - 0s 159us/sample - loss: 2.3075 - accuracy: 0.2907 - val_loss: 2.4625 - val_accuracy: 0.2600
Epoch 4/90
750/750 [==============================] - 0s 168us/sample - loss: 2.1650 - accuracy: 0.3520 - val_loss: 2.4003 - val_accuracy: 0.2800
Epoch 5/90
750/750 [==============================] - 0s 162us/sample - loss: 2.0473 - accuracy: 0.3667 - val_loss: 2.3648 - val_accuracy: 0.2640
Epoch 6/90
750/750 [==============================

查看网络的统计信息

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 8, 64)         640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 4, 64)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 2, 128)         0         
_________________________________________________________________
dropout (Dropout)            (None, 4, 2, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              1

## 3.2 预测测试集

新的数据生成预测

In [20]:
def extract_features(test_dir, file_ext="*.wav"):
    feature = []
    for fn in tqdm(glob.glob(os.path.join(test_dir, file_ext))[:]): # 遍历数据集的所有文件
        X, sample_rate = librosa.load(fn,res_type='kaiser_fast')
        mels = np.mean(librosa.feature.melspectrogram(y=X,sr=sample_rate).T,axis=0) # 计算梅尔频谱(mel spectrogram),并把它作为特征
        feature.extend([mels])
    return feature

保存预测的结果

In [21]:
X_test = extract_features('./Task1_CNN_Baseline_1/test_a/')

100%|██████████| 2000/2000 [05:47<00:00,  5.75it/s]


In [23]:
X_test = np.vstack(X_test)
predictions = model.predict(X_test.reshape(-1, 16, 8, 1))


In [24]:
preds = np.argmax(predictions, axis = 1)
preds = [label_dict_inv[x] for x in preds]

path = glob.glob('./Task1_CNN_Baseline_1/test_a/*.wav')
result = pd.DataFrame({'name':path, 'label': preds})

result['name'] = result['name'].apply(lambda x: x.split('/')[-1])
result.to_csv('submit1.csv',index=None)

In [25]:
!ls ./Task1_CNN_Baseline_1/test_a/*.wav | wc -l

2000


In [27]:
!wc -l submit1.csv

2001 submit1.csv
